<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Python for Algorithmic Trading 

**Chapter 07 &mdash; Working with Real-Time Data and Sockets**

## Visualizing Streaming Data with Plotly

### The Basics

In [1]:
import zmq
from datetime import datetime
import plotly.graph_objects as go

In [2]:
symbol = 'SYMBOL'

In [3]:
fig = go.FigureWidget()
fig.add_scatter()
fig

FigureWidget({
    'data': [{'type': 'scatter', 'uid': '9bdc6757-dd01-49a1-be7d-9d41cf280b90'}], 'layout': {'t…

In [4]:
context = zmq.Context()

In [5]:
socket = context.socket(zmq.SUB)

In [6]:
socket.connect('tcp://0.0.0.0:5555')

In [7]:
socket.setsockopt_string(zmq.SUBSCRIBE, 'SYMBOL')

In [8]:
times = list()
prices = list()

In [9]:
for _ in range(50):
    msg = socket.recv_string()
    t = datetime.now()
    times.append(t)
    _, price = msg.split()
    prices.append(float(price))
    fig.data[0].x = times
    fig.data[0].y = prices

In [10]:
fig = go.FigureWidget()
fig.add_scatter(name='SYMBOL')
fig.add_scatter(name='SMA1', line=dict(width=1, dash='dot'),
                mode='lines+markers')
fig.add_scatter(name='SMA2', line=dict(width=1, dash='dash'),
                mode='lines+markers')
fig

FigureWidget({
    'data': [{'name': 'SYMBOL', 'type': 'scatter', 'uid': '8aa33f31-7e9f-4a50-a563-75211b8241df…

In [11]:
import pandas as pd

In [12]:
df = pd.DataFrame()

In [13]:
for _ in range(75):
    msg = socket.recv_string()
    t = datetime.now()
    sym, price = msg.split()
    df = df.append(pd.DataFrame({sym: float(price)}, index=[t]))
    df['SMA1'] = df[sym].rolling(5).mean()
    df['SMA2'] = df[sym].rolling(10).mean()
    fig.data[0].x = df.index
    fig.data[1].x = df.index
    fig.data[2].x = df.index
    fig.data[0].y = df[sym]
    fig.data[1].y = df['SMA1']
    fig.data[2].y = df['SMA2']

In [14]:
from plotly.subplots import make_subplots

In [15]:
f = make_subplots(rows=3, cols=1, shared_xaxes=True)
f.append_trace(go.Scatter(name='SYMBOL'), row=1, col=1)
f.append_trace(go.Scatter(name='RETURN', line=dict(width=1, dash='dot'),
                mode='lines+markers', marker={'symbol': 'triangle-up'}),
                row=2, col=1)
f.append_trace(go.Scatter(name='MOMENTUM', line=dict(width=1, dash='dash'),
                mode='lines+markers', marker={'symbol': 'x'}), row=3, col=1)
# f.update_layout(height=600)

In [16]:
fig = go.FigureWidget(f)

In [17]:
fig

FigureWidget({
    'data': [{'name': 'SYMBOL',
              'type': 'scatter',
              'uid': 'd8f37a5f…

In [18]:
import numpy as np

In [19]:
df = pd.DataFrame()

In [20]:
for _ in range(75):
    msg = socket.recv_string()
    t = datetime.now()
    sym, price = msg.split()
    df = df.append(pd.DataFrame({sym: float(price)}, index=[t]))
    df['RET'] = np.log(df[sym] / df[sym].shift(1))
    df['MOM'] = df['RET'].rolling(10).mean()
    fig.data[0].x = df.index
    fig.data[1].x = df.index
    fig.data[2].x = df.index
    fig.data[0].y = df[sym]
    fig.data[1].y = df['RET']
    fig.data[2].y = df['MOM']

In [21]:
socket = context.socket(zmq.SUB)

In [22]:
socket.connect('tcp://0.0.0.0:5556')

In [23]:
socket.setsockopt_string(zmq.SUBSCRIBE, '')

In [24]:
for _ in range(5):
    msg = socket.recv_string()
    print(msg)

49.102 19.185 74.012 26.288 69.924 90.448 60.010 30.055
48.869 80.757 13.682 87.540 24.016 93.654 28.022 73.590
87.966 12.753 25.665 84.953 41.787 47.367 11.784 15.018
23.722 19.868 19.881 3.453 58.011 6.336 36.881 7.048
68.677 85.995 83.560 13.433 53.904 29.649 94.214 1.951


In [25]:
fig = go.FigureWidget()
fig.add_bar()
fig

FigureWidget({
    'data': [{'type': 'bar', 'uid': '325ded67-8023-45c7-9d9c-c016d2bc0da0'}], 'layout': {'templ…

In [26]:
x = list('abcdefgh')
fig.data[0].x = x
for _ in range(100):
    msg = socket.recv_string()
    y = msg.split()
    y = [float(n) for n in y]
    fig.data[0].y = y

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>